In [ ]:
import matplotlib,numpy,astropy
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from astroquery.vizier import Vizier
from astropy.table import QTable, join
from astropy.table import Table
from astropy import units
from astropy.coordinates import SkyCoord

In [ ]:
font = {'size'   : 16, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['hatch.linewidth'] = 2

In [ ]:
unresolved_list = [38,49,57,62,89,95]
falsegroup_list = [48,54]
baddata_list = [89]

In [ ]:
HCGs_VLA_HI = Table.read('tables/HCGs_VLA_HI.vo', format='votable')
HCGs_VLA_HI.add_index('HCG')

HCGs_global_HI = Table.read('tables/HCGs_global_HI.vo', format='votable')
HCGs_global_HI.add_index('HCG')

HCGs_HI = astropy.table.join(HCGs_VLA_HI,HCGs_global_HI,keys=['HCG'])
HCGs_HI.add_index('HCG')

HCG_mems = Table.read('tables/HCG_members.vo',format='votable')
HCG_mems.add_index('HCG')

In [ ]:
for HCG in falsegroup_list:
    inx = list(HCGs_VLA_HI['HCG']).index(HCG)
    HCGs_VLA_HI.remove_row(inx)
    inx = list(HCGs_global_HI['HCG']).index(HCG)
    HCGs_global_HI.remove_row(inx)
    inx = list(HCGs_HI['HCG']).index(HCG)
    HCGs_HI.remove_row(inx)
    
for HCG in baddata_list:
    inx = list(HCGs_VLA_HI['HCG']).index(HCG)
    HCGs_VLA_HI.remove_row(inx)
    inx = list(HCGs_global_HI['HCG']).index(HCG)
    HCGs_global_HI.remove_row(inx)
    inx = list(HCGs_HI['HCG']).index(HCG)
    HCGs_HI.remove_row(inx)

In [ ]:
HCGs_HI['Nmem'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='int')

for HCG in HCGs_HI['HCG']:
    HCGs_HI.loc[HCG]['Nmem'] = len(HCG_mems.loc[HCG])

In [ ]:
HCGs_HI['logMHI_p'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')
HCGs_HI['HIphase'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')

HCG_mems_HI = Table.read('tables/HCG_mems_VLA_HI.vo',format='votable')

for HCG in list(HCGs_HI['HCG']):
    
    HCGs_HI.loc[HCG]['logMHI_p'] = r'$'+str(HCGs_HI.loc[HCG]['logMHI_pred_1'])+r' \pm '+str(HCGs_HI.loc[HCG]['e_logMHI_pred_1'])+r'$'
    
    count = numpy.sum(numpy.array(numpy.isfinite(HCG_mems_HI[HCG_mems_HI['HCG'] == HCG]['logMHI']),dtype='int'))
    
    if HCGs_HI.loc[HCG]['ex_frac'] > 0.75 or (hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_HI.loc[HCG]['ex_frac'],'mask')):
        HCGs_HI.loc[HCG]['HIphase'] = '3a'
    elif HCGs_HI.loc[HCG]['ex_frac'] > 0.25:
        HCGs_HI.loc[HCG]['HIphase'] = '2'
    elif count > 1:
        HCGs_HI.loc[HCG]['HIphase'] = '1'
    else:
        HCGs_HI.loc[HCG]['HIphase'] = '3c'
    
    
    if HCG in falsegroup_list:
        HCGs_HI.loc[HCG]['HIphase'] = ''
    elif HCG in unresolved_list and not hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask'):
        HCGs_HI.loc[HCG]['HIphase'] = ''
    #elif hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_HI.loc[HCG]['logMHI_GBT'],'mask'):
    elif hasattr(HCGs_HI.loc[HCG]['logMHI_tot'],'mask'):
        HCGs_HI.loc[HCG]['HIphase'] = '3a_nondet'

In [ ]:
HCGs_WISE = Vizier(catalog='J/ApJ/821/113/table1', columns=['Group', 'Name', 'Class', '_RA', '_DE'], row_limit=-1).query_constraints()[0]

In [ ]:
HCGs_HI['IR_class'] = numpy.array(numpy.zeros(len(HCGs_HI)),dtype='str')

for HCG in list(HCGs_HI['HCG']):
    namestr = 'HCG'+str(HCG).zfill(2)
    
    tmp = HCGs_WISE[HCGs_WISE['Group'] == namestr]
    
    classes = list(tmp['Class'])
    
    N_active = classes.count('Active')
    N_quiet = classes.count('Quiescent')
    N_canyon = classes.count('Canyon')
    N_unknown = int(HCGs_HI.loc[HCG]['Nmem'])-N_active-N_quiet-N_canyon
    
    if float(N_active)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5:
        HCGs_HI.loc[HCG]['IR_class'] = 'Active'
    elif float(N_quiet+N_canyon)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5:
        HCGs_HI.loc[HCG]['IR_class'] = 'Quiescent'
    elif float(N_quiet+N_canyon+N_unknown)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5 or float(N_active+N_unknown)/float(HCGs_HI.loc[HCG]['Nmem']) > 0.5: 
        HCGs_HI.loc[HCG]['IR_class'] = 'Unknown'
    else:
        HCGs_HI.loc[HCG]['IR_class'] = 'Intermediate'

In [ ]:
plt.figure(figsize=(10,8))

counter_nondet = 0

for HCG in list(HCGs_HI['HCG']):
        
    if HCGs_HI.loc[HCG]['HIphase'] == '1':
        col = '#56B4E9'
    elif HCGs_HI.loc[HCG]['HIphase'] == '2':
        col = '#009E73'
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a':
        col = '#D55E00'
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a_nondet':
        col = '#D55E00'
    elif HCGs_HI.loc[HCG]['HIphase'] == '3c':
        col = '#CC79A7'
    else:
        continue
        
    if HCGs_HI.loc[HCG]['IR_class'] == 'Active':
        symbol = '*'
        size = 300
        ecol = 'none'
        fcol = col
        order = 2
    elif HCGs_HI.loc[HCG]['IR_class'] == 'Quiescent':
        symbol = 's'
        size = 125
        ecol = 'none'
        fcol = col
        order = 1
    elif HCGs_HI.loc[HCG]['IR_class'] == 'Unknown':
        symbol = 'o'
        size = 100
        ecol = col
        fcol = 'none'
        order = 0
    elif HCGs_HI.loc[HCG]['IR_class'] == 'Intermediate':
        symbol = 'X'
        size = 150
        ecol = 'none'
        fcol = col
        order = 4
    else:
        print("Error: Unmatched IR classification.")
        break    
    
    if HCGs_HI.loc[HCG]['HIphase'] == '3a_nondet':
        if not hasattr(HCGs_HI.loc[HCG]['logMHI_GBT'],'mask'):
            HIdef_val = HCGs_HI.loc[HCG]['HIdef_GBT']
            plt.scatter(HIdef_val,1.,marker=symbol,s=size,edgecolor=ecol,facecolor=fcol,lw=4,zorder=order)
            if len(HCG_mems.loc[HCG]) == 3:
                plt.scatter(HIdef_val,1.,edgecolor='k',marker='o',s=300, ls='--',facecolor='none')
        else:
            HIdef_val = 1.5
            plt.scatter(HIdef_val,1.+counter_nondet*0.05,marker=symbol,s=size,edgecolor=ecol,facecolor=fcol,lw=4,zorder=order)
            plt.errorbar(HIdef_val,1.+counter_nondet*0.05,c='#D55E00',xlolims=True,xerr=0.05,lw=0,elinewidth=1)
            if len(HCG_mems.loc[HCG]) == 3:
                plt.scatter(HIdef_val,1.+counter_nondet*0.05,edgecolor='k',marker='o',s=300, ls='--',facecolor='none')
            counter_nondet += 1
        continue
    else:
        plt.scatter(HCGs_HI.loc[HCG]['HIdef'],HCGs_HI.loc[HCG]['ex_frac'],marker=symbol,s=size,edgecolor=ecol,facecolor=fcol,lw=4,zorder=order)
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_HI.loc[HCG]['HIdef'],HCGs_HI.loc[HCG]['ex_frac'],edgecolor='k',marker='o',s=300, ls='--',facecolor='none')
    
style = "Simple, tail_width=10, head_width=25, head_length=20"

a1 = patches.FancyArrowPatch(posA=(0., 0.), posB=(0.,0.25),color='#56B4E9', lw=0, arrowstyle=style,alpha=0.25)
a2 = patches.FancyArrowPatch(posA=(0.,0.25), posB=(0.,0.75),color='#009E73', lw=0, arrowstyle=style,alpha=0.25)
a3 = patches.FancyArrowPatch(posA=(0.,0.775), posB=(1.0,1.0),color='#D55E00',connectionstyle="arc3,rad=-.2", lw=0, arrowstyle=style,alpha=0.25)
a4 = patches.FancyArrowPatch(posA=(1.5,0.9), posB=(0.8,0.0125),color='#CC79A7',connectionstyle="arc3,rad=-0.3", lw=0, arrowstyle=style,alpha=0.25)

plt.gca().add_patch(a1)
plt.gca().add_patch(a2)
plt.gca().add_patch(a3)
plt.gca().add_patch(a4)
    
plt.axhline(0.25,lw=1,c='k',ls='dashed')
plt.axhline(0.75,lw=1,c='k',ls='dashed')
plt.ylim(-0.05,1.05)
plt.xlim(-0.5,1.6)

plt.scatter([],[],c='#56B4E9',marker='D',s=100,label='Phase 1')
plt.scatter([],[],c='#009E73',marker='D',s=100,label='Phase 2')
plt.scatter([],[],c='#D55E00',marker='D',s=100,label='Phase 3a')
plt.scatter([],[],c='#CC79A7',marker='D',s=100,label='Phase 3c')
plt.scatter([],[],c='grey',marker='*',s=300,label='Active')
plt.scatter([],[],c='grey',marker='X',s=150,label='Intermediate')
plt.scatter([],[],c='grey',marker='s',s=125,label='Quiescent')
plt.scatter([],[],edgecolor='grey',marker='o',s=100,facecolor='none',lw=4,label='Unknown')
plt.scatter([],[],edgecolor='k',marker='o',s=300, ls='--',facecolor='none',label='Triplet')

plt.legend(loc=7)

plt.xlabel('HI-deficiency [dex]')
plt.ylabel(r'Extended $M_\mathrm{HI}$ / Total $M_\mathrm{HI}$')

plt.savefig('HCG_tidal_frac.png',bbox_inches='tight')
plt.savefig('HCG_tidal_frac.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

ph1_HIdef = []
ph2_HIdef = []
ph3_HIdef = []

for HCG in list(HCGs_HI['HCG']):
        
    if HCGs_HI.loc[HCG]['HIphase'] == '1':
        ph1_HIdef.append(HCGs_HI.loc[HCG]['HIdef'])
    elif HCGs_HI.loc[HCG]['HIphase'] == '2':
        ph2_HIdef.append(HCGs_HI.loc[HCG]['HIdef'])
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a':
        ph3_HIdef.append(HCGs_HI.loc[HCG]['HIdef'])
    elif HCGs_HI.loc[HCG]['HIphase'] == '3a_nondet':
        if not hasattr(HCGs_HI.loc[HCG]['logMHI_GBT'],'mask'):
            ph3_HIdef.append(HCGs_HI.loc[HCG]['HIdef_GBT'])
        else:
            continue
    elif HCGs_HI.loc[HCG]['HIphase'] == '3c': 
        ph3_HIdef.append(HCGs_HI.loc[HCG]['HIdef'])
    else:
        continue
        
defbins = numpy.arange(-0.75,2.01,0.25)

all_HIdef = []
all_HIdef.extend(ph1_HIdef)
all_HIdef.extend(ph2_HIdef)
all_HIdef.extend(ph3_HIdef)

all_binvals, all_edges = numpy.histogram(all_HIdef,defbins)
ph1_binvals, ph1_edges = numpy.histogram(ph1_HIdef,defbins)
ph2_binvals, ph2_edges = numpy.histogram(ph2_HIdef,defbins)
ph3_binvals, ph3_edges = numpy.histogram(ph3_HIdef,defbins)

plt.bar(defbins[:-1],all_binvals,width=0.25,align='edge',color='grey',alpha=0.25,label='All HCGs')
plt.bar(defbins[:-1],ph3_binvals,width=0.25,align='edge',facecolor='none',edgecolor='#E69F00',hatch='/',lw=2,label='Phase 3')
plt.bar(defbins[:-1],ph2_binvals,width=0.25,align='edge',facecolor='none',edgecolor='#009E73',hatch='\\\\',lw=2,label='Phase 2')
plt.bar(defbins[:-1],ph1_binvals,width=0.25,align='edge',facecolor='none',edgecolor='#56B4E9',hatch='+',lw=2,label='Phase 1')

plt.axvline(-0.25,lw=1,ls='dashed',c='k')
plt.axvline(0.25,lw=1,ls='dashed',c='k')

plt.annotate('HI-normal',[0,8.25],ha='center')
plt.annotate('HI-rich',[-0.5,8.25],ha='center')
plt.annotate('HI-poor',[0.5,8.25],ha='center')

plt.legend()

plt.xlim(-0.75,1.75)
plt.ylim(0.,8.75)

plt.xlabel('HI-deficiency [dex]')
plt.ylabel('Count')

plt.savefig('HCG_HIdef_hist.png',bbox_inches='tight')
plt.savefig('HCG_HIdef_hist.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_VLA_HI['logMHI_pred'],HCGs_VLA_HI['logMHI_tot'],c='k',marker='+',s=80,lw=1.5)
m_tmp = numpy.arange(7.35,11.01,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
plt.plot(m_tmp,m_tmp+0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp-0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp+2.*0.13,lw=1,c='k',ls='dotted')
plt.plot(m_tmp,m_tmp-2.*0.13,lw=1,c='k',ls='dotted')

for HCG in HCGs_VLA_HI['HCG']:
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        v_wid_fid = 100. #km/s
        detlim = numpy.log10(3.*(0.001*HCGs_global_HI.loc[HCG]['VLA_rms']/numpy.sqrt(v_wid_fid/abs(HCGs_global_HI.loc[HCG]['chan_wid'])))*v_wid_fid*2.356E5*HCGs_VLA_HI.loc[HCG]['dist']**2.)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',marker='+',s=80.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and not hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='k',marker='+',s=80,lw=1.5)
        
for HCG in HCGs_VLA_HI['HCG']:
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_VLA_HI.loc[HCG]['logMHI_pred'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']-HCGs_VLA_HI.loc[HCG]['HIdef'],
                    edgecolor='k',marker='o',s=150, ls='--',facecolor='none')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (predicted)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (measured)')
plt.xlim(8.75,11.)
plt.ylim(7.35,11.)

plt.savefig('HCG_def.png',bbox_inches='tight')
plt.savefig('HCG_def.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_VLA_HI['logMHI_pred_HG84'],HCGs_VLA_HI['logMHI_tot'],c='g',marker='o',s=50,lw=1.5,label='HG84 relation')
plt.scatter(HCGs_VLA_HI['logMHI_pred'],HCGs_VLA_HI['logMHI_tot'],c='k',marker='+',s=80,lw=1.5,label='Jones+18 relation')
m_tmp = numpy.arange(7.35,11.01,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
plt.plot(m_tmp,m_tmp+0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp-0.13,lw=1,c='k',ls='--')
plt.plot(m_tmp,m_tmp+2.*0.13,lw=1,c='k',ls='dotted')
plt.plot(m_tmp,m_tmp-2.*0.13,lw=1,c='k',ls='dotted')

for HCG in HCGs_VLA_HI['HCG']:
    plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],
             [HCGs_VLA_HI.loc[HCG]['logMHI_tot'],HCGs_VLA_HI.loc[HCG]['logMHI_tot']],lw=1,c='g',ls='--')
    
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        v_wid_fid = 100. #km/s
        detlim = numpy.log10(3.*(0.001*HCGs_global_HI.loc[HCG]['VLA_rms']/numpy.sqrt(v_wid_fid/abs(HCGs_global_HI.loc[HCG]['chan_wid'])))*v_wid_fid*2.356E5*HCGs_VLA_HI.loc[HCG]['dist']**2.)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',marker='+',s=80.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred'],detlim,c='k',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],detlim,c='g',marker='o',s=50.,lw=1.5)
        plt.errorbar(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],detlim,c='g',uplims=True,yerr=0.05,lw=0,elinewidth=1.5)
        plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],[detlim,detlim],lw=1,c='g',ls='--')
        
    if hasattr(HCGs_VLA_HI.loc[HCG]['logMHI_tot'],'mask') and not hasattr(HCGs_global_HI.loc[HCG]['logMHI_GBT'],'mask'):
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='k',marker='+',s=80,lw=1.5)
        plt.scatter(HCGs_global_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_global_HI.loc[HCG]['logMHI_GBT'],c='g',marker='o',s=50,lw=1.5)
        plt.plot([HCGs_VLA_HI.loc[HCG]['logMHI_pred_HG84'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']],
                 [HCGs_global_HI.loc[HCG]['logMHI_GBT'],HCGs_global_HI.loc[HCG]['logMHI_GBT']],lw=1,c='g',ls='--')
        
for HCG in HCGs_VLA_HI['HCG']:
    if len(HCG_mems.loc[HCG]) == 3:
        plt.scatter(HCGs_VLA_HI.loc[HCG]['logMHI_pred'],HCGs_VLA_HI.loc[HCG]['logMHI_pred']-HCGs_VLA_HI.loc[HCG]['HIdef'],
                    edgecolor='k',marker='o',s=150, ls='--',facecolor='none')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (predicted)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (measured)')
plt.xlim(8.75,11.)
plt.ylim(7.35,11.)

plt.legend()

plt.savefig('HCG_def_HG84comp.png',bbox_inches='tight')
plt.savefig('HCG_def_HG84comp.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,8))

plt.scatter(HCGs_global_HI['logMHI_GBT'],HCGs_global_HI['logMHI_VLA_inGBTbeam'],c='k')
m_tmp = numpy.arange(5.5,10.76,0.01)
plt.plot(m_tmp,m_tmp,lw=1,c='k')
#plt.plot(m_tmp,m_tmp-0.25,lw=1,c='k',ls='--')
plt.xlabel('log M_HI (GBT)')
plt.ylabel('log M_HI (VLA weighted to GBT beam)')

plt.xlabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (GBT)')
plt.ylabel(r'$\log\,M_\mathrm{HI}/\mathrm{M_\odot}$ (VLA within GBT beam)')
plt.xlim(8,10.75)
plt.ylim(5.75,10.75)

plt.savefig('HCG_GBT_VLA_HImass.png',bbox_inches='tight')
plt.savefig('HCG_GBT_VLA_HImass.pdf',bbox_inches='tight')